In [1]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
import re
from sklearn.preprocessing import LabelBinarizer
from sklearn.impute import KNNImputer
import os
#pip install pandas-profiling

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'
df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


In [3]:
import xml.etree.ElementTree as ET
def xml_To_df(xml_path):

    xml_data = open(xml_path, 'r').read()  # Read file
    root = ET.XML(xml_data)  # Parse XML
    
    data = []
    cols = []
    for i, child in enumerate(root):
        data.append([subchild.text for subchild in child])
        cols.append(child.tag)
    
    Holidaydf = pd.DataFrame(data).T  # Write in DF and transpose it
    Holidaydf.columns = cols  # Update column names
    Holidaydf=Holidaydf.T
    Holidaydf=Holidaydf.reset_index()
    Holidaydf.drop(columns='index',inplace=True)
    Holidaydf.rename(columns = {0:'justDate',1:'HolidayName'}, inplace = True)
    Holidaydf= Holidaydf.drop(columns= ['HolidayName'])
    Holidaydf['justDate']= pd.to_datetime(Holidaydf['justDate'])
    return Holidaydf

## Initial profiling of the whole dataset

In [4]:
# pp.ProfileReport(df)

## Dropping the ID, Bump and Roundabout based on the initial profiling

In [5]:
# df2 = df.drop(columns=['ID','Bump','Roundabout'])

In [6]:
# pp.ProfileReport(df2)

## Checking significange of each of the categorical features in predicting sevirity:

In [7]:
# categorical_cols= ['Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Stop',
#                    'Amenity', 'Side']
# for c in categorical_cols:
#     print(c, df2[c].value_counts(), "\n")

From the quick investigation above we can see clearly that some of these features are not 
indicative on the sevirity of the accident, so we will remove them.
The columns we will be removing are: "Give_Way", "No_Exit"

In [8]:
# df3= df2.drop(columns= ["Give_Way", "No_Exit"])

In [9]:
# df3.head()

### Converting the Side column cateogories using into 0s, and 1s: 

In [10]:
# print('values before: ', df3.Side.unique())
# # change the side featue into zero for L, one for R
# df3['Side']= df2['Side'].apply(lambda x: 1 if x== 'R' else -1)
# print('values after', df3.Side.unique())
# df3.head()

## Spliting the time stamp data into 4 seprate columns dateyear, datemonth, dateday, datehour: 

In [11]:
# df3['dateyear']= pd.to_datetime(df3['timestamp']).dt.year
# df3['datemonth']= pd.to_datetime(df3['timestamp']).dt.month
# df3['dateday']= pd.to_datetime(df3['timestamp']).dt.day
# df3['datehour']= pd.to_datetime(df3['timestamp']).dt.hour

In [12]:
# df4= df3.drop(columns= ['timestamp'])
# df4.head()

## Finding the correlation between the sevirity, and the rest of the features: 

In [13]:
# for col in df4.columns:
#     print(col+": ", df4['Severity'].corr(df4[col]))

In [14]:
# df4.head()

### Checking correlation of the characteristics of the location and each other 

In [15]:
# pp.ProfileReport(df4)

In [16]:
# for i in df4.columns:
#     for j in df4.columns:
#         print(i+" and "+ j, df4[i].corr(df4[j]))
#     print("\n")

In [17]:
# df4.head()

In [18]:
# df4['timestamp']= df['timestamp']
# df4

### Trial to find the geo location:

In [19]:
# from geopy.geocoders import Nominatim

In [20]:
# geolocator = Nominatim(user_agent="geoapiExercises")

In [21]:
# # lat, long= str(df5['Lat'][0]), str(df5['Lng'][0])
# lat, long

In [22]:
# address= geolocator.reverse(lat+","+long).raw['address']
# zipcode = address.get('postcode')
# zipcode

In [23]:
# df100= df5[:5]
# df100.head()

In [24]:
# def location(df):
#     return geolocator.reverse(str(df['Lat']) + ',' + str(df['Lng'])).raw['address'].get('postcode')

In [25]:
# df5['location']= df5.apply(location, axis= 1)

In [26]:
# df5['location'].head(50)

# Determing days off: 

In [27]:
# df4['timestamp']= pd.to_datetime(df4['timestamp'])
# df4['dayNum']= df4['timestamp'].dt.dayofweek
# df4.head()

In [28]:
# df4['day_off']= df4['dayNum'].apply(lambda x: 1 if (x== 6 or x== 5) else 0)

In [29]:
# df5= df4.drop(columns=['timestamp', 'dayNum'])

In [30]:
# df6.info()

## adding an attribute to represent whether the day was a holiday or not:

In [31]:
# import xml.etree.ElementTree as ET
# xml_data = open('holidays.xml', 'r').read()  # Read file
# root = ET.XML(xml_data)  # Parse XML

# data = []
# cols = []
# for i, child in enumerate(root):
#     data.append([subchild.text for subchild in child])
#     cols.append(child.tag)

# dfHoliday = pd.DataFrame(data).T  # Write in DF and transpose it
# dfHoliday.columns = cols  # Update column names
# dfHoliday=dfHoliday.T
# dfHoliday=dfHoliday.reset_index()
# dfHoliday.drop(columns='index',inplace=True)
# dfHoliday.rename(columns = {0:'justDate',1:'HolidayName'}, inplace = True)
# dfHoliday

In [32]:
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))
weather_df['timestamp'] = pd.to_datetime(weather_df[['Year','Month','Day','Hour']])
weather_df.drop(columns=['Year','Month','Day','Hour'],inplace=True)
weather_df = weather_df.drop_duplicates(subset = ['timestamp'], keep = 'first') 
weather_dfNew= weather_df.drop(columns=['Wind_Chill(F)','Precipitation(in)','Selected'])
weather_dfNew.sort_values('timestamp',inplace=True)
imputer = KNNImputer()
weather_dfNew['Temperature(F)'] = imputer.fit_transform(weather_dfNew[['Temperature(F)']])
weather_dfNew['Humidity(%)'] = imputer.fit_transform(weather_dfNew[['Humidity(%)']])
weather_dfNew['Wind_Speed(mph)'] = imputer.fit_transform(weather_dfNew[['Wind_Speed(mph)']])
weather_dfNew['Visibility(mi)'] = imputer.fit_transform(weather_dfNew[['Visibility(mi)']])
weather_dfNew.dropna(subset=['Weather_Condition'],inplace=True)
weather_dfNew.reset_index(drop=True, inplace=True)

In [33]:
def weather_encoding(df):
    df6 = df
    weather ='!'.join(df6['Weather_Condition'].dropna().unique().tolist())
    weather = np.unique(np.array(re.split(
        "!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather))).tolist()
    df6['Clear'] = np.where(df6['Weather_Condition'].str.contains('Clear|Fair', case=False, na = False), True, False)
    df6['Cloud'] = np.where(df6['Weather_Condition'].str.contains('Cloud|Overcast|Partly Cloudy|Mostly Cloudy|Scattered Clouds|Cloudy', case=False, na = False), True, False)
    df6['Rain'] = np.where(df6['Weather_Condition'].str.contains('Rain|storm|Light Rain|Light Thunderstorms and Rain|Light Drizzle', case=False, na = False), True, False)
    df6['Heavy_Rain'] = np.where(df6['Weather_Condition'].str.contains('Heavy Rain|Rain Shower|Heavy T-Storm|Heavy Thunderstorms', case=False, na = False), True, False)
    df6['Snow'] = np.where(df6['Weather_Condition'].str.contains('Snow|Sleet|Ice', case=False, na = False), True, False)
    df6['Heavy_Snow'] = np.where(df6['Weather_Condition'].str.contains('Heavy Snow|Heavy Sleet|Heavy Ice Pellets|Snow Showers|Squalls', case=False, na = False), True, False)
    df6['Fog'] = np.where(df6['Weather_Condition'].str.contains('Fog|Patches of Fog|Haze|Shallow Fog|Smoke|Mist', case=False, na = False), True, False)
    weather = ['Clear','Cloud','Rain','Heavy_Rain','Snow','Heavy_Snow','Fog']
    for i in weather:
        df6.loc[df6['Weather_Condition'].isnull(),i] = df6.loc[df6['Weather_Condition'].isnull(),'Weather_Condition']
        df6[i] = df6[i].astype('bool')

    df6.loc[:,['Weather_Condition'] + weather]

    df6 = df6.drop(['Weather_Condition'], axis=1)
    def one_column_encoding(row):
        if row['Fog']:
            return 6
        elif row['Heavy_Snow'] == True:
            return 5
        elif row['Snow'] == True:
            return 4
        elif row['Heavy_Rain'] == True:
            return 3
        elif row['Rain'] == True:
            return 2
        elif row['Cloud'] == True:
            return 1
        else:
            return 0
    df6['Condition'] = df6.apply(lambda row: one_column_encoding(row), axis=1)
#     df6.drop(columns=['Fog','Heavy_Snow','Snow','Heavy_Rain','Rain','Cloud'],inplace=True)
    df6.drop(columns=['Condition','Heavy_Snow','Snow','Heavy_Rain'],inplace=True)
    return df6

In [34]:
def data_prep(file_path, xml_path):
    df = pd.read_csv(file_path)
    #droping uninportant functions:
    df= df.drop(columns=['ID', 'Bump', 'Give_Way', 'No_Exit', 'Roundabout', 'Railway', 'Amenity'])
    #extracting year, and month
    df['dateyear']= pd.to_datetime(df['timestamp']).dt.year
    df['datemonth']= pd.to_datetime(df['timestamp']).dt.month
#     df['timestamp_w'] = pd.to_datetime(df["timestamp"])
#     df = df.assign(timestamp_w = df.timestamp_w.dt.floor('H'))
    df['timestamp']= pd.to_datetime(df['timestamp']).dt.date #elimnating the hour:min:sec part from timestamp
    df['dayNum']= pd.to_datetime(df['timestamp']).dt.dayofweek #extracting which day of the week was that
    df['timestamp']= pd.to_datetime(df['timestamp']) #converting timestamp to datetime object
    #create a label to determine weekend
    df['day_off']= df['dayNum'].apply(lambda x: 1 if (x== 6 or x== 5) else 0) 
    Holidaydf= xml_To_df(xml_path)
    
    #Taking the holidays into consideration in the column of day off:
    day_off= list(df['day_off'])
    timestamp= list(df['timestamp'].astype(str))
    Holiday= list(Holidaydf['justDate'].astype(str))
    
    holiday_range= range(len(Holiday))
    time_range= range(len(timestamp))
    for i in holiday_range:
        for j in time_range:
            if timestamp[j]== Holiday[i]:
                day_off[j]= 1
    df['day_off']= pd.Series(day_off)
    df['timestamp'] = pd.to_datetime(pd.read_csv(file_path)['timestamp'])
    df = df.assign(timestamp = df.timestamp.dt.floor('H'))
    df = pd.merge(left=df, right=weather_dfNew, how='left')
    df['Side']= df['Side'].apply(lambda x: 1 if x== 'R' else -1)
    df = weather_encoding(df)
    df= df.drop(columns= ['Crossing', 'Junction', 'Stop', 'Side', 'dayNum', 'timestamp','Humidity(%)'])
    df.dropna(subset=['Temperature(F)','Wind_Speed(mph)','Visibility(mi)'],inplace=True)
    return df

In [35]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'
training_path = os.path.join(dataset_path, 'train.csv')
xml_path = os.path.join(dataset_path, 'holidays.xml')
df = data_prep(training_path,xml_path)

# ML Model: 

In [36]:
from sklearn.model_selection import train_test_split
# df5.drop(columns= ['Lat'], axis= 1)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Severity']) # Try adding `stratify` here
print(df)
X_train = train_df.drop(columns=['Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['Severity'])
y_val = val_df['Severity']


            Lat         Lng  Distance(mi)  Severity  dateyear  datemonth  \
0     37.762150 -122.405660         0.044         2      2016          3   
1     37.719157 -122.448254         0.000         2      2020          5   
2     37.808498 -122.366852         0.000         3      2016          9   
3     37.785930 -122.391080         0.009         1      2020          3   
4     37.719141 -122.448457         0.000         2      2019         10   
...         ...         ...           ...       ...       ...        ...   
6402  37.740630 -122.407930         0.368         3      2017         10   
6403  37.752755 -122.402790         0.639         2      2018         10   
6404  37.726304 -122.446015         0.000         2      2019         10   
6405  37.808090 -122.367211         0.000         3      2019          5   
6406  37.773745 -122.408515         0.000         2      2020          2   

      day_off  Temperature(F)  Wind_Speed(mph)  Visibility(mi)  Clear  Cloud  \
0      

In [37]:
# X_train.head()

In [38]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifie
classifier = RandomForestClassifier(max_depth= 15, random_state=42)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

In [39]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.8369734789391575


## Taking Weather into consideration: 

In [40]:

# weather_df = pd.read_csv('weather-sfcsv.csv')
# severity_df = df.drop(df.columns.difference(['Severity','timestamp']), 1)
# severity_df['timestamp']=pd.to_datetime(df['timestamp'])
# severity_df = severity_df.assign(timestamp = severity_df.timestamp.dt.round('H'))
# severity_df

# # severity_df['Month']=pd.DatetimeIndex(df['timestamp']).month
# # severity_df['Day']=pd.DatetimeIndex(df['timestamp']).day
# # severity_df['Hour']=pd.DatetimeIndex(df['timestamp']).hour
# # result = pd.merge(severity_df, weather_df,  how='inner', on=['Year', 'Month', 'Day', 'Hour'])
# # result
# # weather_df['Weather_Condition'].value_counts()
# # weather_df['timestamp'] = weather_df['Year']+"-"+weather_df['Month']+"-"+weather_df['Day']+" "+weather_df['Hour']+":00:00"
# weather_df['timestamp'] = pd.to_datetime(weather_df[['Year','Month','Day','Hour']])
# weather_df.drop(columns=['Year','Month','Day','Hour'],inplace=True)
# print(weather_df['timestamp'].value_counts())
# # print(severity_df['timestamp'].value_counts())
# weather_df = weather_df.drop_duplicates(subset = ['timestamp'], keep = 'first') 
# print(weather_df['timestamp'].value_counts())
# severity_df.head()

In [41]:
# df5['timestamp']=pd.to_datetime(df['timestamp'])
# df5 = df5.assign(timestamp = df5.timestamp.dt.round('H'))
# df6= df5.merge(weather_df,on='timestamp',how='left')
# df6.sort_values('timestamp',inplace=True)
# pp.ProfileReport(df6)


In [42]:
# df6.fillna(method='bfill',inplace=True)
# pp.ProfileReport(df6)

In [43]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'
test_path = os.path.join(dataset_path, 'test.csv')
xml_path = os.path.join(dataset_path, 'holidays.xml')
df_test = data_prep(test_path,xml_path)

In [44]:
X_test = df_test
# You should update/remove the next line once you change the features used for training
# X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]

y_test_predicted = classifier.predict(X_test)

X_test['Severity'] = y_test_predicted

X_test.head()

,Lat,Lng,Distance(mi),dateyear,datemonth,day_off,Temperature(F),Wind_Speed(mph),Visibility(mi),Clear,Cloud,Rain,Fog,Severity
0,37.786060,-122.390900,0.039,2016,4,0,63.0,10.4,10.0,False,True,False,False,2
1,37.769609,-122.415057,0.202,2020,10,0,65.0,5.0,9.0,True,False,False,False,2
2,37.807495,-122.476021,0.000,2019,9,0,58.0,18.0,10.0,False,True,False,False,2
3,37.761818,-122.405869,0.000,2019,8,0,72.0,17.0,10.0,True,False,False,False,2
4,37.732350,-122.414100,0.670,2018,10,0,57.0,5.8,10.0,False,True,False,False,2


In [45]:
X_test['ID'] = pd.read_csv(test_path)['ID']
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601 entries, 0 to 1600
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Lat              1601 non-null   float64
 1   Lng              1601 non-null   float64
 2   Distance(mi)     1601 non-null   float64
 3   dateyear         1601 non-null   int64  
 4   datemonth        1601 non-null   int64  
 5   day_off          1601 non-null   int64  
 6   Temperature(F)   1601 non-null   float64
 7   Wind_Speed(mph)  1601 non-null   float64
 8   Visibility(mi)   1601 non-null   float64
 9   Clear            1601 non-null   bool   
 10  Cloud            1601 non-null   bool   
 11  Rain             1601 non-null   bool   
 12  Fog              1601 non-null   bool   
 13  Severity         1601 non-null   int64  
 14  ID               1601 non-null   int64  
dtypes: bool(4), float64(6), int64(5)
memory usage: 156.3 KB


In [46]:
# X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
# X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]

# y_test_predicted = classifier.predict(X_test)

# test_df['Severity'] = y_test_predicted

# test_df.head()

In [47]:
X_test[['ID', 'Severity']].to_csv('submission.csv', index=False)